## 4. Prueba e inferencia del modelo

El primer paso es cargar los datos a hacer la predicción y traer toda su información de características del set de datos depurado para poder emplear el modelo para la predicción

In [30]:
import pandas as pd
import joblib

In [31]:
test = pd.read_csv('../data/prueba_op_base_pivot_var_rpta_alt_enmascarado_oot.csv')

In [32]:
informacion_base = pd.read_csv('../datos_depurados/dataset_to_train.csv')

In [33]:
informacion_base.shape

(455372, 46)

In [34]:
test['nit_num_oblig_id'] = test['nit_enmascarado'].astype(str) + '#' + test['num_oblig_orig_enmascarado'].astype(str) + '#' + test['num_oblig_enmascarado'].astype(str)

In [35]:
test.shape

(112549, 5)

In [36]:
dataset = pd.merge(test, informacion_base, on='nit_num_oblig_id', how='inner')
dataset.shape

(68545, 50)

In [37]:
columns_to_delete = ['nit_enmascarado','num_oblig_orig_enmascarado','num_oblig_enmascarado','fecha_var_rpta_alt']
dataset.drop(columns_to_delete, axis=1, inplace=True)

In [38]:
columns_to_delete = ['nit_enmascarado','num_oblig_orig_enmascarado','num_oblig_enmascarado','fecha_var_rpta_alt']
test.drop(columns_to_delete, axis=1, inplace=True)

In [39]:
dataset.head()

,nit_num_oblig_id,Unnamed: 0,y,segmento_x,aplicativo_x,min_mora,max_mora,rango_mora,vlr_obligacion,vlr_vencido,...,prob_propension,prob_alrt_temprana,prob_auto_cura,producto_y,aplicativo_y,segmento_y,pago_total,porc_pago,ajustes_banco,antiguedad
0,397604#973821#106521,239683,1,Personal plus,L,32.0,62.0,b.31-90,16600177.0,944911.0,...,0.359754,0.714111,0.189609,CARTERA MICROCREDITO,L,PERSONAL PLUS,1000000.0,49.0,NO,3855.0
1,397604#973821#106521,324466,1,Personal plus,L,4.0,32.0,a.1-30,16341844.0,944441.0,...,0.359754,0.714111,0.189609,CARTERA MICROCREDITO,L,PERSONAL PLUS,1000000.0,49.0,NO,3855.0
2,397604#973821#106521,324467,1,Personal plus,L,63.0,92.0,b.31-90,16968987.0,2022944.0,...,0.359754,0.714111,0.189609,CARTERA MICROCREDITO,L,PERSONAL PLUS,1000000.0,49.0,NO,3855.0
3,368086#382995#696856,69896,1,Personal,K,5.0,0.0,a.1-30,2876903.0,137345.0,...,0.660659,0.512603,0.225186,TARJETA DE CREDITO,K,PERSONAL,0.0,0.0,NO,2869.0
4,522913#59984#1044995,154941,1,Personal,M,30.0,0.0,a.1-30,793126.0,115672.0,...,0.646185,0.549874,0.297656,TARJETA DE CREDITO,M,PERSONAL,8229.0,10.0,NO,4422.0


Se cargan los binarios del modelo, scaler, y encoder para el pipeline de pre procesamiento y predicción

In [40]:
model = joblib.load('../deployment/artifacts/model_classification.pkl')
scaler = joblib.load('../deployment/artifacts/scaler.pkl')
encoder = joblib.load('../deployment/artifacts/encoder.pkl')

In [41]:
categorical = encoder.feature_names_in_

In [42]:
categorical

array(['segmento_x', 'aplicativo_x', 'rango_mora', 'marca_debito_mora',
       'tipo_cli', 'genero_cli', 'declarante', 'cli_actualizado',
       'subsegm', 'ciiu', 'producto_y', 'aplicativo_y', 'segmento_y',
       'ajustes_banco'], dtype=object)

In [43]:
def transform(df):
    encoded_df = pd.DataFrame(
        encoder.transform(df[categorical]),
        columns=encoder.get_feature_names_out(categorical)
    )
    df = pd.concat([df.drop(categorical, axis=1), encoded_df], axis=1)

    return df

In [44]:
df_test = transform(dataset)

In [45]:
df_test.head()

,nit_num_oblig_id,Unnamed: 0,y,min_mora,max_mora,vlr_obligacion,vlr_vencido,endeudamiento,cant_alter_posibles,cant_gestiones,...,aplicativo_y_V,segmento_y_EMPRESARIAL,segmento_y_MICROPYME,segmento_y_PERSONAL,segmento_y_PERSONAL PLUS,segmento_y_PREFERENCIAL,segmento_y_PYMES,segmento_y_SOCIAL,ajustes_banco_AJUSTES,ajustes_banco_NO
0,397604#973821#106521,239683,1,32.0,62.0,16600177.0,944911.0,55567085.0,1,30.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,397604#973821#106521,324466,1,4.0,32.0,16341844.0,944441.0,54836111.0,1,78.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,397604#973821#106521,324467,1,63.0,92.0,16968987.0,2022944.0,56387251.0,1,18.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,368086#382995#696856,69896,1,5.0,0.0,2876903.0,137345.0,2878067.0,2,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,522913#59984#1044995,154941,1,30.0,0.0,793126.0,115672.0,2397330.0,1,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df_test_sample = df_test.head(100)
df_test_sample.drop(['Unnamed: 0'],axis=1, inplace=True)
df_test_sample.to_csv('datos_testing.csv')

In [26]:
df_test.drop(['Unnamed: 0', 'y'],axis=1, inplace=True)

In [27]:
df_test_pred = df_test.drop('nit_num_oblig_id',axis=1)

Set de datos listo para su estimación

In [28]:
df_test_pred.head()

,min_mora,max_mora,vlr_obligacion,vlr_vencido,endeudamiento,cant_alter_posibles,cant_gestiones,rpc,promesas_cumplidas,cant_promesas_cumplidas_binario,...,aplicativo_y_V,segmento_y_EMPRESARIAL,segmento_y_MICROPYME,segmento_y_PERSONAL,segmento_y_PERSONAL PLUS,segmento_y_PREFERENCIAL,segmento_y_PYMES,segmento_y_SOCIAL,ajustes_banco_AJUSTES,ajustes_banco_NO
0,32.0,62.0,16600177.0,944911.0,55567085.0,1,30.0,1.0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,4.0,32.0,16341844.0,944441.0,54836111.0,1,78.0,1.0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,63.0,92.0,16968987.0,2022944.0,56387251.0,1,18.0,1.0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,5.0,0.0,2876903.0,137345.0,2878067.0,2,0.0,1.0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,30.0,0.0,793126.0,115672.0,2397330.0,1,4.0,1.0,2,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


Predicción de la variable objetivo y la probabilidad de que el resultado estimado sea 1

In [153]:
df_test['var_rpta_alt'] = model.predict(df_test_pred)

In [156]:
probabilities = model.predict_proba(df_test_pred)


In [157]:
df_test['Prob_uno'] = probabilities[:, 1]

In [158]:
df_test.head()

,nit_num_oblig_id,min_mora,max_mora,vlr_obligacion,vlr_vencido,endeudamiento,cant_alter_posibles,cant_gestiones,rpc,promesas_cumplidas,...,segmento_y_MICROPYME,segmento_y_PERSONAL,segmento_y_PERSONAL PLUS,segmento_y_PREFERENCIAL,segmento_y_PYMES,segmento_y_SOCIAL,ajustes_banco_AJUSTES,ajustes_banco_NO,var_rpta_alt,Prob_uno
0,397604#973821#106521,32.0,62.0,16600177.0,944911.0,55567085.0,1,30.0,1.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0.821296
1,397604#973821#106521,4.0,32.0,16341844.0,944441.0,54836111.0,1,78.0,1.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0,0.258026
2,397604#973821#106521,63.0,92.0,16968987.0,2022944.0,56387251.0,1,18.0,1.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,0.634636
3,368086#382995#696856,5.0,0.0,2876903.0,137345.0,2878067.0,2,0.0,1.0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.965385
4,522913#59984#1044995,30.0,0.0,793126.0,115672.0,2397330.0,1,4.0,1.0,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.995298


In [163]:
prediction = df_test[['nit_num_oblig_id', 'var_rpta_alt', 'Prob_uno']]

Resultado obtenido

In [164]:
prediction.head()

,nit_num_oblig_id,var_rpta_alt,Prob_uno
0,397604#973821#106521,1,0.821296
1,397604#973821#106521,0,0.258026
2,397604#973821#106521,1,0.634636
3,368086#382995#696856,1,0.965385
4,522913#59984#1044995,1,0.995298


Se guarda el dataset

In [165]:
prediction.to_csv('submission.csv')

### Prueba del pipeline funcionando con datos de un usuario en JSON

In [4]:
import json
with open('../datos_depurados/example/input.json', 'r', encoding='utf-8') as archivo:
    datos = json.load(archivo)

In [5]:
from prediction import predict
from preprocessing import clean_data

In [3]:
df = clean_data(datos)
predict(df)

/home/sergioq2/AI/Bancolombia/codigo/preprocessing.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ciiu'].fillna('NO REGISTRA', inplace=True)
/home/sergioq2/AI/Bancolombia/codigo/preprocessing.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

{'prediction': 1, 'probability': 0.8}